# Intro to HPC: Solar Cell project

Project: Solar Power for Affordable Housing through Computational Design of Low-Cost/High-Efficiency Solar Cells.
Author: AlvaroVM [https://alvarovm.github.io](http://alvarovm.github.io)

Version: 0.0.2

## Part 6 Machine Learning with Gaussian Process Regression 

In this notebook we will use Gaussian Process Regression (GPR) to predict molecular optical activity, particularly to predict the spectral peak that corresponds to the first excitation energy. In this exercise we will need to the DyeDB and the data we agregated in previous notebooks.

We would use GPR algorithm implemented in the GPflow code. This implementation relies on Tensorflow, which is a popular package for artificial intelligence and is portable among different computer architectures. Tensorflow could use GPUs and accelerate regression and inference. One advantage of fast regression is that we could optimize hyper parameters and create models that fit better the data.



In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import rdkit
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit import DataStructs 
from rdkit.Chem import Draw
from rdkit.Chem.rdMolDescriptors import  GetHashedMorganFingerprint
from rdkit.DataStructs import ConvertToNumpyArray
from rdkit.Chem.Draw import IPythonConsole
print(f'        RDkit version : {rdkit.__version__}')

from tqdm.autonotebook import tqdm

from collections import OrderedDict

# utils
from collections import OrderedDict
from tqdm.autonotebook import tqdm
# scientific python

#import seaborn as sns
import sklearn
import sklearn.metrics
import sklearn.model_selection
import sklearn.preprocessing
import sklearn.ensemble
import scipy.stats

import tensorflow as tf
print(f'   Tensorflow version : {tf.__version__}')
print("{} GPU(s) recognized by tensorflow:".format(len(tf.config.list_physical_devices('GPU'))), tf.config.list_physical_devices('GPU'))


import gpflow
from gpflow.utilities import print_summary
print(f'       GPflow version : {gpflow.__version__}')
tqdm.pandas()


#### Test GPU 

In [ ]:
# This cell gives us a couple options for debugging Tensorflow.
# It is the first code cell, because it must be run, before the TensorFlow library is imported and it is most convenient to import all modules in the next cell
# To enable this debugging, you must change one of the debugging flags to True and run this cell *before* importing running later cells
# Currently this is only done manually from the notebook, but could be included as a JSON setting in the future if desirable

# import os
#os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'
#export CUDA_VISIBLE_DEVICE=
#export CUDA_VISIBLE_DEVICE=0
#export CUDA_VISIBLE_DEVICE=0,1

# Disable GPUS
# os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

# print(f'   Tensorflow version : {tf.__version__}')
# print("{} GPU(s) recognized by tensorflow:".format(len(tf.config.list_physical_devices('GPU'))), tf.config.list_physical_devices('GPU'))


In [ ]:
def rmse(pred, target):
    """
    Input Args:
    pred: predictions numpy array
    target :True label numpy array
    return : rmse 
    """
    return np.sqrt(((pred - target) ** 2).mean())

In [ ]:
from rdkit.Chem import MolFromSmiles as smi2mol
from rdkit.Chem.rdMolDescriptors import  GetHashedMorganFingerprint,GetMorganFingerprintAsBitVect


from rdkit.Chem import MolFromSmiles as smi2mol
def canon_smiles(smi):
    try:
        m = smi2mol(smi)
        if m is None:
            #print('hola'+smi)
            return False
        else:
            return Chem.MolToSmiles(m, isomericSmiles=True, canonical=True)
    except:
        return False
    
def applyMorganFP(m,**kwargs):
    fptype='bit'
    
    if 'fptype' in kwargs:
        fptype=kwargs['fptype']
    if 'fp_args' in kwargs:
        fp_args=kwargs['fp_args']     
    #fp_args = self.meta_data['fp_args']
    #fptype = self.meta_data['fptype']
    arr = np.zeros((1,))
    if fptype == 'bit': 
        arr = np.zeros((1,))
        #ConvertToNumpyArray(GetHashedMorganFingerprint(m, **fp_args), arr)
        try:
            arr = np.array(GetMorganFingerprintAsBitVect(m, **fp_args))
        except:
            print(Chem.MolToSmiles(m))
    elif fptype == 'count':
        #arr = np.zeros((1,))
        ConvertToNumpyArray(GetHashedMorganFingerprint(m, **fp_args), arr)
    return arr

### Load data

In [ ]:
df = pd.read_pickle('../data/../data/extended_db_Zindo_Nov_2019_V5_cannfp_clust_lem.pkl').fillna(value = 0)
print('Column names: {}'.format(str(df.columns.tolist())))
print('Table Shape: {}'.format(df.shape))
#df.head(2)

### Clean data

In [ ]:
#df=df[df['lambda_tddft (nm)']>0]
df=df[df['lambda_sTDA (nm)']>0]
#df=df[df['cluster']>-1]
print('Table Shape: {}'.format(df.shape))

### Sample randomly a faction of the data and reset indexes

In [ ]:
df = df.sample(n=1000, replace=True, random_state=1)
df = df.reset_index()

In [ ]:
df.head()

In [ ]:
getonly=['NHOHCount', 'NOCount', 'NumAliphaticCarbocycles',
                           'NumAliphaticHeterocycles', 'NumAliphaticRings',
                           'NumAromaticCarbocycles', 'NumAromaticHeterocycles',
                           'NumAromaticRings', 'NumHAcceptors', 'NumHDonors', 
                           'NumHeteroatoms', 'NumRadicalElectrons', 'NumRotatableBonds',
                           'NumSaturatedCarbocycles', 'NumSaturatedHeterocycles', 
                           'NumSaturatedRings', 'NumValenceElectrons',
'qed','TPSA', 'MolLogP', 'MolMR','BalabanJ', 'BertzCT',
'fr_Al_OH', 'fr_Al_OH_noTert', 'fr_ArN', 'fr_Ar_COO',
                            'fr_Ar_N', 'fr_Ar_NH', 'fr_Ar_OH', 'fr_COO', 'fr_COO2', 
                            'fr_C_O', 'fr_C_O_noCOO', 'fr_C_S', 'fr_HOCCN', 'fr_Imine',
                            'fr_NH0', 'fr_NH1', 'fr_NH2', 'fr_N_O', 'fr_Ndealkylation1',
                            'fr_Ndealkylation2', 'fr_Nhpyrrole', 'fr_SH', 'fr_aldehyde',
                            'fr_alkyl_carbamate', 'fr_alkyl_halide', 'fr_allylic_oxid', 'fr_amide',
                            'fr_amidine', 'fr_aniline', 'fr_aryl_methyl', 'fr_azide', 'fr_azo',
                            'fr_barbitur', 'fr_benzene', 'fr_benzodiazepine', 'fr_bicyclic',
                            'fr_diazo', 'fr_dihydropyridine', 'fr_epoxide', 'fr_ester', 'fr_ether',
                            'fr_furan', 'fr_guanido', 'fr_halogen', 'fr_hdrzine', 'fr_hdrzone', 
                            'fr_imidazole', 'fr_imide', 'fr_isocyan', 'fr_isothiocyan', 'fr_ketone',
                            'fr_ketone_Topliss', 'fr_lactam', 'fr_lactone', 'fr_methoxy', 'fr_morpholine',
                            'fr_nitrile', 'fr_nitro', 'fr_nitro_arom', 'fr_nitro_arom_nonortho', 
                            'fr_nitroso', 'fr_oxazole', 'fr_oxime', 'fr_para_hydroxylation', 
                            'fr_phenol', 'fr_phenol_noOrthoHbond', 'fr_phos_acid', 'fr_phos_ester', 
                            'fr_piperdine', 'fr_piperzine', 'fr_priamide', 'fr_prisulfonamd',
                            'fr_pyridine', 'fr_quatN', 'fr_sulfide', 'fr_sulfonamd', 'fr_sulfone',
                            'fr_term_acetylene', 'fr_tetrazole', 'fr_thiazole', 'fr_thiocyan', 
                            'fr_thiophene', 'fr_unbrch_alkane', 'fr_urea',
                            'MolWt','MolLogP']
getonly=['NHOHCount', 'NOCount', 'NumAliphaticCarbocycles',
                           'NumAliphaticHeterocycles', 'NumAliphaticRings',
                           'NumAromaticCarbocycles', 'NumAromaticHeterocycles',
                           'NumAromaticRings', 'NumHAcceptors', 'NumHDonors', 
                           'NumHeteroatoms', 'NumRadicalElectrons', 'NumRotatableBonds',
                           'NumSaturatedCarbocycles', 'NumSaturatedHeterocycles', 
                           'NumSaturatedRings', 'NumValenceElectrons',
                            ]

In [ ]:
# import inspect
# import rdkit.Chem.Descriptors as Descriptors


# calc_props = OrderedDict(inspect.getmembers(Descriptors, inspect.isfunction))
# for key in list(calc_props.keys()):
#     if key.startswith('_'):
#         del calc_props[key]
#     else:
#         thisnot=False
#         for myprop in getonly:
#             if myprop == key:
#                 thisnot=True
#         if not thisnot:
#             del calc_props[key]


# def calc_all(df,calc_props,smiles_col='smiles'):
#     df['mol'] = df[smiles_col].apply(Chem.MolFromSmiles)
#     for key,val in calc_props.items():
#         df[key] = df['mol'].apply(val)
#         #df[key] = df['mol'].swifter.apply(val)
#     return df
# print('Found {} molecular descriptors in RDKIT'.format(len(calc_props)))
# #calc_props

In [ ]:
#from gpmol.features.fingerprint_features import canon_smiles
#from gpmol.features.fingerprints3D_features import geo3d_sanity_check
#from gpmol.utils.mol import canon_smiles



print('Before properties: {}'.format(str(df.columns.tolist())))
#df = geo3d_sanity_check(df)
#df['smiles'] = df['smi'].apply(canon_smiles)
#df['smiles'] = df['SMI']
#calc_all(df,calc_props)
#df['mol'] = df['smiles'].apply(Chem.MolFromSmiles)
#df['morganfps']=df['mol'].apply(applyMorganFP,fp_args={'radius':4, 'nBits':2048})
#print(df.shape)
#df.head(1)

In [ ]:
#calc_all(df,calc_props)
#print('After properties: {}'.format(str(df.columns.tolist())))

### Define tools for feature selection

In [ ]:
def rf_feature_mask(x, y, k):
   rf = sklearn.ensemble.RandomForestRegressor()
   rf.fit(x,y)
   indices = (-rf.feature_importances_).argsort()[:k]
   mask = np.zeros_like(rf.feature_importances_)
   mask[indices] = 1.0
   return mask.astype(bool)

class IdentityTransformer(sklearn.preprocessing.FunctionTransformer):
    
    def __init__(self):
        super().__init__(lambda x:x, lambda x:x)

class GPFeature():
    
    def __init__(self, label, x, y, n_features=None, preproc=None):
        self.label = label
        n_features = n_features or x.shape[-1]
        x_feats = x.shape[-1]
        if n_features > x_feats:
            raise ValueError(f'n_features={n_features} larger than x.shape[-1] {x_feats}')
        elif n_features < x_feats:
            mask = rf_feature_mask(x, y, n_features)
        else:
            mask = np.ones(x_feats).astype(bool)
        self.mask = mask
        preproc = preproc or IdentityTransformer()
        self.preproc = preproc.fit(x[:,self.mask])
        self.ndim = sum(self.mask)
        self.active_dims = np.arange(self.ndim)
    
    def transform(self, x):
        return self.preproc.transform(x[:,self.mask]).astype(np.float64)
    
def evaluate_result(y_true, y_pred, fold=None):
    result = OrderedDict()
    result['r2'] = sklearn.metrics.r2_score(y_true,y_pred)
    result['rmse'] = np.sqrt(sklearn.metrics.mean_squared_error(y_true,y_pred))
    result['mae'] = np.sqrt(sklearn.metrics.mean_absolute_error(y_true,y_pred))
    result['r'] = scipy.stats.pearsonr(y_test.ravel(),y_pred.ravel())[0]
    if fold:
        result['cv'] = fold
    return result

#### Train/Test split

In [ ]:
indices=df.index.tolist()
train_index,test_index = sklearn.model_selection.train_test_split(indices, test_size=.30)
print(len(train_index),len(test_index))

#### Standarize data

In [ ]:
target='lambda_sTDA (nm)'
y_true = df[target].values.reshape(-1,1).astype(np.float64)
y_preproc = sklearn.preprocessing.StandardScaler()
y_train = y_preproc.fit_transform(y_true[train_index])
y_test = y_true[test_index]
print(y_train.shape, y_test.shape)

#### Select fingerprint and select the most relevant features that will be used for training

In [ ]:

fp = np.vstack(df['morganfps-b'])
print(fp.shape)
#n_features = 200
fp_feat = GPFeature('fp', fp[train_index], y_train, 200)
fp_train = fp_feat.transform(fp[train_index]) 
fp_test = fp_feat.transform(fp[test_index]) 
print(fp_train.shape,fp_test.shape)

#### Select Properties

In [ ]:
other_features = getonly
x_tmp = df[other_features].values
#n_features = 4
chem_feat = GPFeature('chemoinformatic', x_tmp[train_index],y_train, 4, sklearn.preprocessing.StandardScaler())
chem_train = chem_feat.transform(x_tmp[train_index])
chem_test = chem_feat.transform(x_tmp[test_index])
print(chem_train.shape,chem_test.shape)

#### Combine fingerprint and descriptors

In [ ]:
x_train = np.hstack((fp_train,chem_train))
x_test = np.hstack((fp_test,chem_test))
print(x_train.shape,x_test.shape)

### GP regression

#### Building Kernel

In [ ]:
last_dim = 0
fp_kernel = gpflow.kernels.RBF(active_dims=fp_feat.active_dims+last_dim)
last_dim  = fp_kernel.active_dims[-1]+1
chem_kernel = gpflow.kernels.RBF(active_dims=chem_feat.active_dims+last_dim)
kernel = fp_kernel + chem_kernel
print_summary(kernel)

#### Specify kernel

In [ ]:
model = gpflow.models.GPR(data=(x_train, y_train), kernel=kernel, mean_function=None)

#### Optimize model

In [ ]:
opt = gpflow.optimizers.Scipy()

def objective():
    return - model.log_marginal_likelihood()

opt_logs = opt.minimize(objective,
                        model.trainable_variables,
                        options=dict(maxiter=500))
print_summary(model)

#### Predict and scale (inverse), print statistics of the model

In [ ]:

y_pred, y_std  = model.predict_y(x_test)
y_pred = y_preproc.inverse_transform(y_pred.numpy())
results = [evaluate_result(y_test, y_pred)]
pd.DataFrame(results)

#### Plot the comparison between predicted and true

In [ ]:
plt.scatter(y_test,y_pred)
plt.xlabel('Y Test')
plt.ylabel('Y Pred')
plt.title(f"GPR {target}")